<a href="https://colab.research.google.com/github/YolandaMDavis/wildtrack-iqa/blob/task-amenability/generate_image_feature_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Feature File Generation

This notebook is used to create a non-reference iqa data set from the images provided by Wildtrack. Taking a directory of images, the tool will calculate all IQA measure for each imaage and store them in files (presplit for training validation and test) along with image metadata.

In [ ]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'WildTrack_Raw.zip'

In [ ]:
# clone the repository and move data file
!git clone https://github.com/YolandaMDavis/wildtrack-iqa.git
!mv WildTrack_Raw.zip wildtrack-iqa/.
%cd "wildtrack-iqa"
!git checkout multiclass-switch

fatal: destination path 'wildtrack-iqa' already exists and is not an empty directory.
/content/wildtrack-iqa
M	general_image_statistics.py
Already on 'multiclass-switch'
Your branch is up to date with 'origin/multiclass-switch'.


In [ ]:
from zipfile import ZipFile
# extract zip file
with ZipFile('WildTrack_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [ ]:
# set directory variables and 
import os

import numpy as np
from scipy import stats
import torch.nn as nn
import random

root_dir = '/content/wildtrack-iqa'
data_dir = root_dir + '/RAW'
image_reference_file_suffix = '_image_references.csv'

%cd "/content/wildtrack-iqa"

/content/wildtrack-iqa


In [ ]:
from torch.utils import data
import csv
import random
import torch
import yaml
import cv2
from brisque import brisque
from niqe import niqe
from piqe import piqe
from general_image_statistics import image_brightness, blurry
from time import process_time

from torch.utils.data import Dataset


def get_image_row(image_reference):
    file_path = os.path.join(image_reference[0], image_reference[3])
    print("Processing image for features:" + file_path)
    im_features = cv2.imread(file_path)
    im_features = cv2.cvtColor(im_features, cv2.COLOR_BGR2RGB)
    brisque_features = brisque(im_features)
    n_score = niqe(im_features)
    p_score,_,_,_ = piqe(im_features)
    brightness = image_brightness(im_features)
    blurriness = blurry(im_features)
    other_scores = [n_score, p_score,  brightness, blurriness]
    iqa_features = np.append(brisque_features,np.array(other_scores))
    iqa_features = tuple(iqa_features.tolist())
    return image_reference + iqa_features

def generate_file(dataset):
  ref_file_name = root_dir + '/' + dataset[0] + image_reference_file_suffix
  with open(ref_file_name, 'w', newline='') as csvfile:
      image_ref_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
      header_columns = ['image_location','species','animal_class','image_name','subj_score']
      brisque_columns = ['feature_' + str(i) for i in range(0,36)]
      header_columns+=brisque_columns
      header_columns+=['n_score','p_score', 'brightness', 'blurriness']
      image_ref_writer.writerow(header_columns)
      image_ref_writer.writerows(dataset[1])

def generate_data_files(sample_percentage=1):

    image_reference_list = []

    subdirectories = list(os.walk(data_dir, topdown=False))[:-1]
    for subdir in subdirectories:
        image_location = subdir[0]
        images = subdir[2]
        species_rating = image_location.rsplit('/', 1)[-1].replace('_', ' ')
        score = int(species_rating.rsplit(' ', 1)[-1])
        species_class = species_rating.rsplit(' ', 1)[:-1][0]
        if len(species_class.rsplit(' ', 1)) > 1:
            species = species_class.rsplit(' ')[0]
            animal_class = ' '.join(species_class.rsplit(' ')[1:])
        else:
            animal_class = 'Unknown'
            species = species_class

        for image in images:
            image_reference = (image_location, species, animal_class, image, score)
            image_reference_list.append(image_reference)
        

    # shuffle then split
    seed = 1234
    random.Random(seed).shuffle(image_reference_list)

    train_index = int(len(image_reference_list) *0.6 *sample_percentage)
    test_index = -int(len(image_reference_list) * 0.2 * sample_percentage) 
    val_index = test_index * 2

    training = image_reference_list[:train_index]
    validation = image_reference_list[val_index:test_index]
    testing = image_reference_list[test_index:]

    training_features = [get_image_row(image_reference) for image_reference in training]
    validation_features = [get_image_row(image_reference) for image_reference in validation]
    test_features = [get_image_row(image_reference) for image_reference in testing]

    # generated reference splits
    for dataset in [('training', training_features), ('validation', validation_features), ('testing', test_features)]:
        dataset_type = dataset[0]
        print("Processing File: " + dataset_type)
        generate_file(dataset)
        shutil.copy(dataset_type + '_image_references.csv', PARENT_DIR + '/data/' + dataset_type + '_image_references.csv') 

In [ ]:
# take a percentage of the full data set as a training/test/validation sample
sample_size=1
generate_data_files(sample_size)

Processing image for features:/content/wildtrack-iqa/RAW/Otter_Eurasian_3/11fd443a38b611eb8b6300155dc81a1c.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Leopard_African_4/193de3a6986211eb843100155db6dfa2.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Otter_Eurasian_4/21334450c0fb11ea82a50242ac1c0002.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Otter_Eurasian_4/0060f4b066bf11ebafe200155d00ce15.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Rhino_White_3/eced4f74d3dd11ea91590242ac1c0002.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Otter_Eurasian_3/e98354d237ec11ebaaca00155ddc05b7.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Jaguar_4/4ad4f2fafad411ea9d950242ac110002.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Tiger_Bengal_5/02791c10d3de11ea91590242ac1c0002.jpg
Processing image for features:/content/wildtrack-iqa/RAW/Otter_Eurasian_3/b183588c09c211eb9d950242ac110002.jpg
Processing im